In [1]:
# Libraries
import open3d as o3d
import viser
import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
#@title Importing the Clusters with It's radiances
slices = torch.load("/workspace/FruitProposal/attachment/RadianceCloud/slice_points.pt")

planes = slices['planes']
points = slices['points']
rgbs   = slices['rgb']

print(slices.keys())

dict_keys(['points', 'plane', 'rgb'])


In [ ]:
server = viser.ViserServer()


